In [1]:
# imports
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

# Title.akas.tsv.gz

- **titleId (string):** A `tconst`, an alphanumeric unique identifier of the title
- **ordering (integer):** A number to uniquely identify rows for a given `titleId`
- **title (string):** The localized title
- **region (string):** The region for this version of the title
- **language (string):** The language of the title
- **types (array):** Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- **attributes (array):** Additional terms to describe this alternative title, not enumerated
- **isOriginalTitle (boolean):** 0: not original title; 1: origin
# Title.basics.tsv.gz

- **tconst (string):** Alphanumeric unique identifier of the title
- **titleType (string):** The type/format of the title (e.g., movie, short, tvseries, tvepisode, video, etc)
- **primaryTitle (string):** The more popular title / the title used by the filmmakers on promotional materials at the point of release
- **originalTitle (string):** Original title, in the original language
- **isAdult (boolean):** 0: non-adult title; 1: adult title
- **startYear (YYYY):** Represents the release year of a title. In the case of TV Series, it is the series start year
- **endYear (YYYY):** TV Series end year. ‘\N’ for all other title types
- **runtimeMinutes:** Primary runtime of the title, in minutes
- **genres (string array):** Includes up to three genres associated with

# Title.ratings.tsv.gz

- **tconst (string):** Alphanumeric unique identifier of the title
- **averageRating:** Weighted average of all the individual user ratings
- **numVotes:** Number of votes the title has received
 the title
al title


[Source](https://datasets.imdbws.com/)


In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [11]:
#Replace "\N" with np.nan:
basics.replace(r'\N', np.nan, inplace=True)


In [10]:
#Eliminate movies that are null for runtimeMinutes:
basics = basics.dropna(subset=['runtimeMinutes'])

In [8]:
#Eliminate movies that are null for genre:
basics = basics.dropna(subset=['genres'])


In [12]:
#Keep only titleType==Movie:
basics = basics[basics['titleType'] == 'movie']


In [25]:
# Remove duplicates
basics = basics.drop_duplicates()

In [26]:
# Check for and remove duplicates
basics = basics[~basics.duplicated()]

# Print the resulting DataFrame
print(basics)

             tconst titleType  \
13081     tt0013274     movie   
34800     tt0035423     movie   
61111     tt0062336     movie   
67485     tt0068865     movie   
67663     tt0069049     movie   
...             ...       ...   
10246115  tt9916538     movie   
10246156  tt9916622     movie   
10246183  tt9916680     movie   
10246205  tt9916730     movie   
10246215  tt9916754     movie   

                                               primaryTitle  \
13081                           Istoriya grazhdanskoy voyny   
34800                                        Kate & Leopold   
61111     The Tango of the Widower and Its Distorting Mi...   
67485                                   Lives of Performers   
67663                            The Other Side of the Wind   
...                                                     ...   
10246115                                Kuambil Lagi Hatiku   
10246156        Rodolpho Teóphilo - O Legado de um Pioneiro   
10246183  De la ilusión al desconcie

In [27]:
# Convert 'startYear' to numeric
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce')
# Convert 'startYear' to integer
basics['startYear'] = basics['startYear'].astype(int)
# Keep startYear 2000-2022
basics = basics[(basics['startYear'].notnull()) & (basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]


In [29]:
# Eliminate movies that include "Documentary" in genre
basics = basics[~basics['genres'].str.contains('Documentary')]


In [35]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61111,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67485,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67663,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
80548,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror


In [34]:
#Keep only US movies (Use AKAs table):
us_movies = basics.merge(akas[akas['region'] == 'US'], left_on='tconst', right_on='titleId')


In [50]:
us_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",tt0035423,39,Kate and Leopold,US,\N,\N,alternative spelling,0
1,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",tt0035423,42,Kate & Leopold,US,\N,imdbDisplay,\N,0
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama,tt0062336,6,The Tango of the Widower and Its Distorting Mi...,US,\N,imdbDisplay,\N,0
3,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama,tt0068865,3,Lives of Performers,US,\N,imdbDisplay,\N,0
4,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,tt0069049,3,The Other Side of the Wind,US,\N,imdbDisplay,\N,0


In [32]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [33]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [36]:
#Replace "\N" with np.nan:
akas.replace("\\N", np.nan, inplace=True)

In [47]:
# Keep only US movies
df_us_movies = akas[akas['region'] == 'US']


In [49]:
df_us_movies.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [51]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [52]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1893
3,tt0000004,5.5,178
4,tt0000005,6.2,2678


In [54]:
#Replace "\N" with np.nan (if any)
ratings.replace("\\N", np.nan, inplace=True)

In [64]:

# Keep only US movies
us_ratings = ratings.merge(akas[akas['region'] == 'US'], left_on='tconst', right_on='titleId', how='inner')



In [62]:
print("Columns in 'ratings':", ratings.columns)
print("Columns in 'akas':", df_us_movies.columns)



Columns in 'ratings': Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
Columns in 'akas': Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')


In [67]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")


[]

In [72]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [73]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [79]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)


In [70]:
us_ratings.to_csv('Data/us_ratings.csv.gz', index=False, compression='gzip')


In [71]:
us_movies.to_csv('Data/us_movies.csv.gz', index=False, compression='gzip')

In [74]:

df_us_movies.to_csv('Data/df_us_movies.csv.gz', index=False, compression='gzip')

In [76]:
us_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569750 entries, 0 to 569749
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           569750 non-null  object 
 1   averageRating    569750 non-null  float64
 2   numVotes         569750 non-null  int64  
 3   titleId          569750 non-null  object 
 4   ordering         569750 non-null  int64  
 5   title            569750 non-null  object 
 6   region           569750 non-null  object 
 7   language         2856 non-null    object 
 8   types            374499 non-null  object 
 9   attributes       30293 non-null   object 
 10  isOriginalTitle  569750 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 52.2+ MB


In [77]:
us_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97161 entries, 0 to 97160
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tconst           97161 non-null  object
 1   titleType        97161 non-null  object
 2   primaryTitle     97161 non-null  object
 3   originalTitle    97161 non-null  object
 4   isAdult          97161 non-null  object
 5   startYear        97161 non-null  int32 
 6   endYear          0 non-null      object
 7   runtimeMinutes   97161 non-null  object
 8   genres           97161 non-null  object
 9   titleId          97161 non-null  object
 10  ordering         97161 non-null  int64 
 11  title            97161 non-null  object
 12  region           97161 non-null  object
 13  language         97161 non-null  object
 14  types            97161 non-null  object
 15  attributes       97161 non-null  object
 16  isOriginalTitle  97161 non-null  object
dtypes: int32(1), int64(1), object(1

In [78]:
df_us_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472163 entries, 5 to 37516964
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472163 non-null  object
 1   ordering         1472163 non-null  int64 
 2   title            1472163 non-null  object
 3   region           1472163 non-null  object
 4   language         4168 non-null     object
 5   types            984662 non-null   object
 6   attributes       47623 non-null    object
 7   isOriginalTitle  1470822 non-null  object
dtypes: int64(1), object(7)
memory usage: 101.1+ MB
